### iter()

Python has a built-in function called iter(). When you pass it a collection, you get back an iterator object:

In [1]:
numbers = [7, 4, 11, 3]
iter(numbers)

In [2]:
numbers_iter=iter(numbers)
for num in numbers_iter:
    print(num)

7
4
11
3


You don't normally need to do this. If you instead write:

    for num in numbers:

what python effectively does under the hood is call **iter()** on that collection.
<br>

How does iter() actually get the iterator? It can do this in several ways, but on relies on a magic method called **\_\_iter\_\_()**

Python makes a distinction between objects which are iterators, and objects which are iterable. 

We say an object is iterable if and only if you can pass it to iter(), and get a ready-to-use iterator. If that object has an __iter__ method, iter() will call it to get the iterator.


#### for loop

A for loop is the most common way to step through a sequence. But sometimes your code needs to step through in a more fine-grained way. 

For this, use the built-in function *next()*. You normally call it with a single argument, which is an iterator. Each time you call it, next(my_iterator)
fetches and returns the next element:

In [4]:
names= ["John","Hunt","Deckol"]

names_it = iter(names)

print(next(names_it))
print(next(names_it))
print(next(names_it))


John
Hunt
Deckol


In [5]:
#What happens? if you call print(next(names_it)) again? 
print(next(names_it))


StopIteration: 

next() will raise a special built-in exception, called StopIteration:

When using next() yourself, you can provide a second argument, for the default value. If you do, next() will return that instead of raising StopIteration at the end:

In [6]:
names = ["Tom", "Shelly", "Garth"]
new_names_it = iter(names)
print(next(new_names_it,"Rick"))
print(next(new_names_it,"Rick"))
print(next(new_names_it,"Rick"))
print(next(new_names_it,"Rick"))


Tom
Shelly
Garth
Rick


In [7]:
print(next(new_names_it))

StopIteration: 

In [8]:
print(next(new_names_it,"Errick"))

Errick


### Memory footprint of sequence

In [9]:
def fetch_squares(max_root):
    squares=[]
    for n in range(max_root):
        squares.append(n**2)
    return squares

MAX = 5

for square in fetch_squares(MAX):
    print(square)

0
1
4
9
16


This works fine. But what if **MAX** is not just 5 but 1000000000? Your memory footprint is pointelessly dreadful. There's gonna be a massive list, uses it once and throw it away. <br><br>

To top it off, the second loop (for square in fetch_squares(1000000000)) cannot even start until the entire list of squares has been fully calculated. <br><br>

The solution is to create an iterator to start with, lazily computing each value only when needed. Then each cycle through the loop happens just in time.

In [11]:
class SquaresIterator:
    def __init__(self,max_root_value):
        self.max_root_value = max_root_value
        self.current_root_value=0
    def __iter__(self):
        return self
    def __next__(self):
        if self.current_root_value >= self.max_root_value:
            raise StopIteration
        square_value = self.current_root_value**2
        self.current_root_value +=1
        return square_value

#you can use it like this
for square in SquaresIterator(5):
    print(square)

0
1
4
9
16


### It's way too mouthful. There's got to be a better way; and here comes generator.

In [12]:
def gen_nums():
    n = 0
    while n < 4:
        yield n
        n += 1

for num in gen_nums():
    print(num)

0
1
2
3


In [18]:
#When you call gen_nums() like a function, it immediately returns a generator object:

sequence = gen_nums()
type(sequence)

generator

A function is a generator function if and only if it uses "yield" instead of "return".

In [19]:
print(next(sequence))
print(next(sequence))
print(next(sequence))
print(next(sequence))
print(next(sequence,"Bicky"))

0
1
2
3
Bicky


So far, this is much like a regular function. But the next time next() is called - or, equivalently, the next time through the for loop - the function doesn’t start at the beginning again. 

It starts on the line AFTER the yield statement. Let's look at the gen_nums() again:

In [20]:
def gen_nums():
    n = 0
    while n < 4:
        yield n
        n += 1

gen_num() is more than a function; but coroutine;<br>

Coroutin is like a function except it has several possible entry points.<br>
It starts with the first line, like a normal function. But when it "returns",<br>
the coroutine isn't existing, so much as ***pausing***. 

Subsequent calls with next() start at that yield statement again, right where it left off; the re-entry point is the line after the yield statement. And that's the key:

    Each yield statement simultaneously defines an exit point, and a re-entry
    point.
    
Plus, In fact, you can have multiple yield statements in a generator:

In [21]:
def gen_extra_nums():
    n = 0
    while n < 4:
        yield n
        n += 1
    yield 42 # Second yield

In [22]:
for num in gen_extra_nums():
    print(num)

0
1
2
3
42


Let’s revisit the earlier example, of cycling through a sequence of squares.<br>

This is how we first did it.


In [26]:
def fetch_squares(max_root):
    squares = []
    for n in range(max_root):
        squares.append(n**2)
    return squares
MAX = 5
print()
for square in fetch_squares(MAX):
    print(square)


0
1
4
9
16


In [27]:
#see if you can write a gen_squares generator function that accomplishes the same thing.

def fetch_squares(max_root):
    for n in range(max_root):
        yield n**2

for square in fetch_squares(5):
    print(square)

0
1
4
9
16


let’s look at the SquaresIterator class again: